In [1]:
from src.data_local import MergedDataPreprocessing
from cashe import get_cashed_input

## HJH Dataset
loaded on 23/5/2024
loaded on 12/08/2024

In [2]:
df_original = get_cashed_input()

In [3]:
df_original['CREATION_DATE'].sort_values(ascending=False)

164579   2024-05-30 11:46:53
164571   2024-05-30 11:46:53
164549   2024-05-30 11:46:53
164569   2024-05-30 11:46:53
164557   2024-05-30 11:46:53
                 ...        
398361   2024-01-01 12:38:36
398363   2024-01-01 12:38:36
398365   2024-01-01 12:38:36
398360   2024-01-01 12:38:12
398359   2024-01-01 12:33:25
Name: CREATION_DATE, Length: 207106, dtype: datetime64[ns]

In [4]:
df_original.VISIT_ID.value_counts()

VISIT_ID
211067    100
241410     98
202428     95
245528     93
237374     91
         ... 
217366      1
217367      1
217369      1
217371      1
194135      1
Name: count, Length: 33392, dtype: int64

In [4]:
preprocessing = MergedDataPreprocessing(df_original)

In [6]:
for col in df_original.columns:
    print(col,":",df_original[col].dtype)

CREATED_BY_USER : int64
CREATION_DATE : datetime64[ns]
VISIT_ID : int64
NET_WITH_VAT : float64
SERVICE_DESCRIPTION : object
QTY : float64
LINE_CLAIMED_AMOUNT : float64
LINE_CLAIMED_AMOUNT_SAR : float64
CO_PAY : float64
CO_INSURANCE : float64
LINE_ITEM_DISCOUNT : float64
NET_AMOUNT : float64
NET_VAT_AMOUNT : float64
PATIENT_VAT_AMOUNT : float64
VAT_PERCENTAGE : float64
TREATMENT_TYPE_INDICATOR : int64
SERVICE_TYPE : object
DURATION : float64
QTY_STOCKED_UOM : int64
UNIT_PRICE : float64
OASIS_IOS_DESCRIPTION : object
SEQUENCE_NO : int64
UNIT_PRICE_STOCKED_UOM : float64
UNIT_PRICE_NET : float64
DISCOUNT_PERCENTAGE : float64
NOTES : object
OUTCOME : object
APPROVED_QUNATITY : float64
REQUEST_DATE : datetime64[ns]
VISIT_NO : object
VISIT_DATE : datetime64[ns]
EMERGENCY_INDICATOR : object
PROVIDER_DEPARTMENT_CODE : int64
PROVIDER_DEPARTMENT : object
DOCTOR_SPECIALTY_CODE : float64
DOCTOR_CODE : int64
PATIENT_NO : int64
EPISODE_NO : int64
PATIENT_ID : int64
MEMBERSHIP_NO : object
PATIENT_ID_T

In [7]:
df_original.SERVICE_TYPE

398359            services
398360      oral-health-op
398361            services
398365    medication-codes
398364          procedures
                ...       
164577     medical-devices
164576             imaging
164575             imaging
164573          procedures
164579     medical-devices
Name: SERVICE_TYPE, Length: 207106, dtype: object

In [8]:
df_original.isna().sum()

CREATED_BY_USER           0
CREATION_DATE             0
VISIT_ID                  0
NET_WITH_VAT              0
SERVICE_DESCRIPTION       0
                       ... 
BATCH_PERIOD_START        0
BATCH_PERIOD_END          0
Episode_Key            4003
ICD10                  4003
Diagnosis_Key          4003
Length: 96, dtype: int64

In [9]:
df_original.OUTCOME.value_counts()

OUTCOME
APPROVED    107601
REJECTED     85234
PARTIAL      14271
Name: count, dtype: int64

In [10]:
df_original.PATIENT_AGE.dtype

dtype('int64')

In [14]:
df_original.sort_values(['CREATION_DATE'])

,CREATED_BY_USER,CREATION_DATE,VISIT_ID,NET_WITH_VAT,SERVICE_DESCRIPTION,QTY,LINE_CLAIMED_AMOUNT,LINE_CLAIMED_AMOUNT_SAR,CO_PAY,CO_INSURANCE,...,ATTACHMENT_TYPE,NEW_BORN,PATIENT_IDENTITY_TYPE,BILLABLE_PERIOD_START,BILLABLE_PERIOD_END,BATCH_PERIOD_START,BATCH_PERIOD_END,Episode_Key,ICD10,Diagnosis_Key
398359,9643,2024-01-01 12:33:25,194135,134.40,Consultant Consultation,1.0,120.00,120.00,24.00,96.00,...,FILE,N,IQ,2023-12-07 00:00:00,2023-12-21 00:00:00,2023-12-01,2023-12-10,1_1346536-2,J93,1_1111225808
398360,9643,2024-01-01 12:38:12,194136,138.00,Consultation,1.0,120.00,120.00,0.00,120.00,...,FILE,N,H,2023-12-06 00:00:00,2023-12-20 00:00:00,2023-12-01,2023-12-10,1_1343530-4,K02,1_1111225570
398361,9643,2024-01-01 12:38:36,194137,12.98,Unlisted Code,1.0,16.13,16.13,0.00,16.13,...,FILE,N,H,2023-12-03 00:00:00,2023-12-04 00:00:00,2023-12-01,2023-12-10,1_1343731-2,N80.1,1_1111228814
398365,9643,2024-01-01 12:38:36,194137,1.56,SPASMOPAN 20MG AMP.,1.0,1.69,1.69,0.31,1.38,...,FILE,N,H,2023-12-03 00:00:00,2023-12-04 00:00:00,2023-12-01,2023-12-10,1_1343731-2,N80.1,1_1111228814
398364,9643,2024-01-01 12:38:36,194137,1427.30,Therapeutic ultrasound,1.0,1791.68,1791.68,100.00,1691.68,...,FILE,N,H,2023-12-03 00:00:00,2023-12-04 00:00:00,2023-12-01,2023-12-10,1_1343731-2,N80.1,1_1111228814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164555,10335,2024-05-30 11:46:53,252432,8.40,"Hypodermic needle, single-use",4.0,12.00,12.00,0.00,12.00,...,FILE,Y,H,2024-05-07 11:39:50,2024-05-11 14:29:56,2024-05-01,2024-05-15,1_1370946-1,Z38,1_111260843
164557,10335,2024-05-30 11:46:53,252432,61.53,Unlisted Procedure Code,1.0,87.90,87.90,0.00,87.90,...,FILE,Y,H,2024-05-07 11:39:50,2024-05-11 14:29:56,2024-05-01,2024-05-15,1_1370946-1,Z38,1_111260843
164569,10335,2024-05-30 11:46:53,252432,26.49,WATER FOR INJECTION,1.0,37.84,37.84,0.00,37.84,...,FILE,Y,H,2024-05-07 11:39:50,2024-05-11 14:29:56,2024-05-01,2024-05-15,1_1370946-1,Z38,1_111260843
164566,10335,2024-05-30 11:46:53,252432,148.00,I.V ADMIXTURE SERVICE,2.0,160.00,160.00,0.00,160.00,...,FILE,Y,H,2024-05-07 11:39:50,2024-05-11 14:29:56,2024-05-01,2024-05-15,1_1370946-1,Z38,1_111260843


In [5]:
df_train, df_test = preprocessing.train_test_split_time(id_column='CREATION_DATE',test_size=0.2)

In [6]:
len(df_test)/ len(df_train)

0.25000603558581397

In [7]:
ls1 = list(df_train.sort_values(by='VISIT_ID')['VISIT_ID'])
ls2 = list(df_test.sort_values(by='VISIT_ID')['VISIT_ID'])
intersection = list(set(ls1) & set(ls2))
intersection ## to find the repeated transaction id between them

[235723]

In [8]:
len(df_train) / len(df_test)

3.999903432958331

In [9]:
path_date = 'data/HJH/12-06-2024'; train_prq = path_date + '/train.parquet';test_prq = path_date + '/test.parquet'

df_train.to_parquet(train_prq)
df_test.to_parquet(test_prq)

In [11]:
df_test.CREATION_DATE

322046   2024-04-06 13:18:08
322044   2024-04-06 13:18:08
322048   2024-04-06 13:18:09
322047   2024-04-06 13:18:09
304945   2024-04-06 13:18:25
                 ...        
164555   2024-05-30 11:46:53
164557   2024-05-30 11:46:53
164569   2024-05-30 11:46:53
164566   2024-05-30 11:46:53
164579   2024-05-30 11:46:53
Name: CREATION_DATE, Length: 41422, dtype: datetime64[ns]

In [10]:
df_train.ICD10

398359      J93
398360      K02
398361    N80.1
398365    N80.1
398364    N80.1
          ...  
301417    R07.4
301418    R07.4
301414    R07.4
316581      K21
322045    R07.4
Name: ICD10, Length: 165684, dtype: object